In [ ]:
# 7-2 에서 relu 로 바꾸고 NN
# 0.949

# model

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# image size = 28 * 28 = 784
# 0~9 digits
image_size = 784
layer_size = 256
nb_classes = 10
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, image_size])
Y = tf.placeholder(tf.float32, [None, nb_classes])

W1 = tf.Variable(tf.random_normal([image_size, layer_size]))
b1 = tf.Variable(tf.random_normal([layer_size]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([layer_size, layer_size]))
b2 = tf.Variable(tf.random_normal([layer_size]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([layer_size, nb_classes]))
b3 = tf.Variable(tf.random_normal([nb_classes]))
hypothesis = tf.matmul(L2, W3) + b3

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# learning

# 15 epoch
# 100개 씩 batch
epochs = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        cost_val, _ = sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys})
        avg_cost += cost_val / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "Cost:", "{:.9f}".format(avg_cost))
    
# Accuracy

# sess.run(accuracy, ...) 와 동일 방법임
print("Accuracy:", accuracy.eval(session=sess, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

# xavier initialization tensorflow 구글링하면 많이 나옴

In [ ]:
# NN + xavier
# 0.9786

# model

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# image size = 28 * 28 = 784
# 0~9 digits
image_size = 784
layer_size = 256
nb_classes = 10
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, image_size])
Y = tf.placeholder(tf.float32, [None, nb_classes])

W1 = tf.get_variable("W1", shape=[image_size, layer_size], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([layer_size]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("W2", shape=[layer_size, layer_size], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([layer_size]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("W3", shape=[layer_size, nb_classes], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([nb_classes]))
hypothesis = tf.matmul(L2, W3) + b3

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# learning

# 15 epoch
# 100개 씩 batch
epochs = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        cost_val, _ = sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys})
        avg_cost += cost_val / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "Cost:", "{:.9f}".format(avg_cost))
    
# Accuracy

# sess.run(accuracy, ...) 와 동일 방법임
print("Accuracy:", accuracy.eval(session=sess, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

# DNN 가즈아
* Accuracy 떨어짐.. Overfitting

# Dropout

In [1]:
# DNN + xavier + dropout
# 0.9825

# model

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# image size = 28 * 28 = 784
# 0~9 digits
image_size = 784
layer_size = 512
nb_classes = 10
learning_rate = 0.001

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, image_size])
Y = tf.placeholder(tf.float32, [None, nb_classes])

W1 = tf.get_variable("W1", shape=[image_size, layer_size], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([layer_size]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W2", shape=[layer_size, layer_size], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([layer_size]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[layer_size, layer_size], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([layer_size]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.get_variable("W4", shape=[layer_size, layer_size], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([layer_size]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.get_variable("W5", shape=[layer_size, nb_classes], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([nb_classes]))
hypothesis = tf.matmul(L4, W5) + b5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
is_correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# learning

# 15 epoch
# 100개 씩 batch
epochs = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        cost_val, _ = sess.run([cost, optimizer], feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7})
        avg_cost += cost_val / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "Cost:", "{:.9f}".format(avg_cost))
    
# Accuracy

# sess.run(accuracy, ...) 와 동일 방법임
print("Accuracy:", accuracy.eval(session=sess, feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch: 0001 Cost: 0.459254506
Epoch: 0002 Cost: 0.172762187
Epoch: 0003 Cost: 0.130621731
Epoch: 0004 Cost: 0.106213285
Epoch: 0005 Cost: 0.093703559
Epoch: 0006 Cost: 0.083242041
Epoch: 0007 Cost: 0.076489095
Epoch: 0008 Cost: 0.067064687
Epoch: 0009 Cost: 0.064010754
Epoch: 0010 Cost: 0.058632498
Epoch: 0011 Cost: 0.058118020
Epoch: 0012 Cost: 0.047046929
Epoch: 0013 Cost: 0.051048783
Epoch: 0014 Cost: 0.047755462
Epoch: 0015 Cost: 0.046533416
Accuracy: 0.9825


# Optimizers
* GradientDescentOptimizer
* AdamOptimizer
* 기타등등...